In [2]:
import numpy as np

import numpy as np
from scipy.linalg import kron
from numpy import identity

def tensor(a, i, N, p=2):
    """
    TENSOR takes matrix a and implements it as the ith qubit
    Embeds the matrix a in a p^N dimensional Hilbert space of N qudits
    in the ith position in the tensor product.
    It is assumed a is a pxp 1-qudit matrix.

    Parameters:
    - a: pxp Unitary matrix to operate on a single qudit
    - i: Position in the tensor product, indexing from 1
    - N: Number of qudits in the Hilbert space
    - p: Degree of freedom per site (default is 2 for qubits)

    Returns:
    - A: Resulting matrix after tensor product
    """
    dim1 = int(p**(i-1))
    dim2 = int(p**(N-i))

    A = kron(kron(np.identity(dim1), a), np.identity(dim2))
    return A


Commutation Relation Result:
[[ 0.70710678  0.        ]
 [ 0.         -0.70710678]]


Here we are make the JW transformation. We have $\chi_{2n-1} = (\prod_{j=1}^{n-1}σ^z_j)σ_n^x$ and $\chi_{2n} = (\prod_{j=1}^{n-1}σ^z_j)σ_n^y$. To explain the code we build the string on Zs and then based on even or odd position we apply the corresponding Pauli metrix. after we applied the matrices we update the stirng of Zs for the next opration.

In [ ]:
def majorana(N):
    # MAJORANA creates a representation of N majorana fermions

    if N % 2 != 0:
        raise ValueError("N must be even")

    # SU(2) Matrices
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)

    # Output of majoranas here
    dim = int(2**(N/2))
    Xi = np.zeros((dim, dim, N), dtype=complex)

    # Growing chain of Zs, start with 0
    Zs = np.eye(dim, dtype=complex)

    for i in range(1, N+1):
        # The X or Y at the end
        if i % 2 == 1:
            xi = tensor(X, (i+1)//2, N//2)
        else:
            xi = tensor(Y, i//2, N//2)

        Xi[:, :, i-1] = np.dot(Zs, xi)

        # Build an Z string
        if i % 2 == 0:
            Zs = np.dot(Zs, tensor(Z, i//2, N//2))

    #Kitaev's normalization
    Xi = Xi / np.sqrt(2)


    return Xi


Now we build the SYK Hamiltonian

In [ ]:
import numpy as np

def kitaev_H(J, Xi, N):
    """
    KITAEV_H returns Hamiltonian for Kitaev's chaotic Hamiltonian.
    Specifically, this builds a nonlocal system of N majorana fermions with
    random interactions chosen from a Gaussian with variance J^2.

    Let Xi_i be a Majorana, where i runs over 1..N, N = 2*log2(length(Xi)),
    and {Xi_i, Xi_j} = delta_ij. The Hamiltonian is given by

    H =  \Sum_{i<j<k<l} J_{ijkl} Xi_i Xi_j Xi_k Xi_l

    N.B. HL comes with a negative sign.
    N.B.2. The Hilbert space is 2^(N/2) dimensional

    Parameters:
    - J: The spread of the coupling distribution.
    - Xi: The Majorana fermions.

    Returns:
    - H: The Hamiltonian matrix.
    """

    if N % 2 != 0:
        raise ValueError("N must be even")

    if N < 4:
        raise ValueError("N must be greater than 4")

    # build Hamiltonian
    H_length = 2**(N//2)
    H = np.zeros((H_length, H_length), dtype=complex)

    var_J_jklm = np.math.factorial(3) * J**2 / ((N-1)*(N-2)*(N-3))
    sigma = np.sqrt(var_J_jklm)

    # Iterate over combinations (j, k, l, m)
    for j in range(1, N+1):
        for k in range(j+1, N+1):
            for l in range(k+1, N+1):
                for m in range(l+1, N+1):
                    J_jklm = np.random.randn() * sigma
                    H += J_jklm * np.dot(np.dot(np.dot(Xi[:, :, j-1], Xi[:, :, k-1]), Xi[:, :, l-1]), Xi[:, :, m-1])

    return H
